In [1]:
import scipy.optimize as optimize

import pandas_datareader as pdr
import datetime 
import pandas as pd
import numpy as np

import pandas_datareader.data as web
import requests_cache
import matplotlib.pyplot as plt
from datetime import date

# Constants

# Current TBILL rate
TBILL_PERCENT = 1.66

In [22]:
from prefutils import *

# initialize web cache
SESSION = init_fetch_session()

# read the FloatingReset database file
floats = pd.read_csv('./floats.csv')

# filter some tickers, remove some columns
floats = filter_floating_reset_list(floats)


In [57]:
tprefs = update_data_frame_with_prices_and_drop_reference(floats,SESSION)

In [58]:
tprefs = update_dataframe_with_market_spread(tprefs)

In [101]:
#tpm = update_dataframe_with_rating_averages(tprefs)
#tpm.head(5)

In [118]:
# market spread deltas: model increasing or decreasing levels of fear
SPREAD_DELTAS = [ 0.2, 0, -0.1, -.3]

# define 4 scenarios, with corresponding probabilities and names.
# Name: {tbillrate, number of years, probability}
TBILL_SCN = { 
    "Constant" :  (1.66, 1, 0.15),
    "SlightDrop": (1.40, 1, 0.45),
    "BigDrop":    (1.10, 1, 0.25) ,
    "Panic":      (0.70, 1, 0.15)}


foo = create_tbill_scenarios_per_market_spreads(tprefs, TBILL_SCN, SPREAD_DELTAS)
foo.head(3)


,Ticker,Rating,Spread,Price,MSpread,EffMSpread,MSpread_Delta,Expected_Gain,Constant,PriceConstant,SlightDrop,PriceSlightDrop,BigDrop,PriceBigDrop,Panic,PricePanic
0,AZP.PR.C,P5H,418.0,18.60,6.1895,6.3895,0.2,0.028322,0.053644,18.137773,0.037835,17.908723,0.018531,17.624675,-0.009218,17.208548
1,AIM.PR.B,P4H,375.0,16.33,6.6223,6.8223,0.2,0.024802,0.059245,15.944968,0.037730,15.658636,0.011476,15.304899,-0.026218,14.789360
2,BPO.PR.S,P3I,348.0,19.15,5.0502,5.2502,0.2,0.011571,0.038157,18.595699,0.021687,18.345313,0.001354,18.030928,-0.028332,17.562435


In [147]:

ranked = do_the_ranking(foo,TBILL_SCN,10)
ranked.head(3)

,Ticker,TotalRankSum,AvgExpectedGain
2,AZP.PR.C,16,0.061220
24,SLF.PR.K,16,0.059426
0,AIM.PR.B,16,0.055606


In [148]:
ranked = ranked.merge(floats[['Ticker','Rating']],how='inner',on='Ticker')
ranked.head(10)

,Ticker,TotalRankSum,AvgExpectedGain,Rating
0,AZP.PR.C,16,0.061220,P5H
1,SLF.PR.K,16,0.059426,P1L
2,AIM.PR.B,16,0.055606,P4H
3,BPO.PR.S,16,0.050383,P3I
4,FFH.PR.D,12,0.044539,P3I
5,FFH.PR.J,16,0.037913,P3I
6,IFC.PR.D,10,0.035190,P2I
7,BRF.PR.B,5,0.032922,P3H
8,ALA.PR.B,8,0.032465,P3L
9,FFH.PR.H,3,0.030728,P3I


In [159]:
# Now, for hedging. For each security, it needs to make a 5% hurdle.
# So for example, if IFC.PR.D is only expected to make 3.5%, then it needs a 1.5% hedge gain to compensate.

def sumproduct_scenario(scn) :
    sum = 0;
    for (a,_,b) in scn.values() : sum += a*b       
    return sum

expected_tbill_rate = sumproduct_scenario(TBILL_SCN)
# print(expected_tbill_rate)
expected_drop = TBILL_PERCENT - expected_tbill_rate
# So if we drop by 0.4%, we want a 1.5% hedge gain then hedge ratio is 1.5/0.4 = 3.75. Would have to
# hedge $3.75 for each $ of IFC.PR.D
expected_drop

# but actually, should we reduce the amount of net hedges based on shares that exceed the hurdle?



1.259


0.401

In [165]:
GAIN_HURDLE=0.05
ranked['GainToHedgePercent'] = [max(GAIN_HURDLE-x,0)*100 for x in ranked['AvgExpectedGain']]
ranked['HedgeRatio'] = [x/expected_drop for x in ranked['GainToHedgePercent']]
ranked.head(10)

,Ticker,TotalRankSum,AvgExpectedGain,Rating,GainToHedgePercent,HedgeRatio
0,AZP.PR.C,16,0.061220,P5H,0.000000,0.000000
1,SLF.PR.K,16,0.059426,P1L,0.000000,0.000000
2,AIM.PR.B,16,0.055606,P4H,0.000000,0.000000
3,BPO.PR.S,16,0.050383,P3I,0.000000,0.000000
4,FFH.PR.D,12,0.044539,P3I,0.546065,1.361758
5,FFH.PR.J,16,0.037913,P3I,1.208677,3.014158
6,IFC.PR.D,10,0.035190,P2I,1.480980,3.693216
7,BRF.PR.B,5,0.032922,P3H,1.707758,4.258748
8,ALA.PR.B,8,0.032465,P3L,1.753545,4.372929
9,FFH.PR.H,3,0.030728,P3I,1.927211,4.806012
